In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from urllib.request import urlopen
import time
from selenium.webdriver.common.by import By
import requests
from selenium.webdriver.common.keys import Keys
from datetime import datetime
import xlwt
import json
from datetime import datetime, timedelta

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://ebook.kyobobook.co.kr/dig/etc/landing/best")

index = "kyobo_best_100_"

present_date = str(datetime.utcnow() + timedelta(hours=9))[:10]
book_list = []

genre_dict = { 
        18 : "경제경영", # 
        19 : "자기계발", # 경제/비즈니스
        20 : "시/에세이", # 
        21 : "인문", # 
        22 : "종교", #  x
        23 : "소설", # 
        24 : "국어/외국어", # 
        25 : "정치/사회", #  x
        26 : "역사/문화", #  x
        27 : "과학/공학", # 
        28 : "IT/프로그래밍", # 
        29 : "건강/의학", #  
        30 : "가정/생활/요리", # 
        31 : "여행/취미", # 
        32 : "예술/대중문화", # 
        33 : "유아",  # x
        34 : "아동", # x
        35 : "청소년", # x
        36 : "교재/수험서", #
        37 : "외국도서", #
        38 : "매거진" # x
}

sm_100_genre_dict = {
    1 : "문학", #23
    2 : "에세이/산문", #20
    3 : "인문", #21
    4 : "경제/비즈니스", #18,19
    5 : "자연/과학", #27
    6 : "컴퓨터/인터넷", #28
    7 : "외국어", #24
    8 : "가정/생활", #29,30, 31, 32
    9 : "강의지원도서", #36
    10 : "국외ebook" #38
}

#100개씩 보기 선택
driver.find_element(By.ID,"ebookLanding_pageCnt-button").click()
time.sleep(0.5)
    
driver.find_element(By.ID,"ui-id-58").click()
time.sleep(0.5)

bsObject = BeautifulSoup(driver.page_source, 'html.parser')
dates = bsObject.select_one('#ui-id-42').text # 오늘날짜

# 장르 선택
for genre_num in range(18,39):
    driver.find_element(By.ID,"ebookLanding_msc-button").click()
    time.sleep(0.5)
    
    driver.find_element(By.ID,f"ui-id-{genre_num}").click()
    time.sleep(0.5)
    
    bsObject = BeautifulSoup(driver.page_source, 'html.parser')
    
    books = bsObject.select('div.prodDt')
    rank = 0
    for book in books:
        
        rank = rank + 1# 순위
        
        title = book.select_one('strong > h3 > a').text  #책 제목
        
        author = book.select_one('p.prodDt_info > span').text  #저자
        
        publisher = book.select('p.prodDt_info > span')[1].text #출판사
        
        date = book.select('p.prodDt_info > span')[2].text  #등록일
        
        price =book.select_one('div.prodDt_price span b').text # 가격
        
        if(book.select_one('div.prodDt_review > p > b') == None) : #만약 별점이 없으면 0으로
            review = 0
            
        else :
            review = book.select_one('div.prodDt_review > p > b').text #별점
            
        genre=genre_dict[genre_num]
        book_list.append([title,rank,author,publisher,date,price,review,genre,genre,present_date])

df = pd.DataFrame(book_list,columns=["kyobo_subject","kyobo_rank","kyobo_writer","kyobo_publisher","kyobo_date","kyobo_price","kyobo_review","kyobo_genre","smu_genre","present_date"])

# 장르 상명대에게 맞추기
df2 = df.replace({'smu_genre' : {'경제경영':'경제/비즈니스','자기계발':'경제/비즈니스','시/에세이':'에세이/산문','소설':'문학','국어/외국어':'외국어','과학/공학':'자연/과학','IT/프로그래밍':'컴퓨터/인터넷','가정/생활/요리':'가정/생활','여행/취미':'가정/생활','예술/대중문화':'가정/생활','교재/수험서':'수험서/자격증','외국도서':'국외ebook'}})

#데이터 전처리
df2['kyobo_date'] = df2['kyobo_date'].str.replace('.', '')
df2['kyobo_date'] = df2['kyobo_date'].str.slice(0,8)
df2['kyobo_price'] = df2['kyobo_price'].str.replace(',', '')
#df2['kyobo_date'] = pd.to_datetime(df2['kyobo_date'])

df3 = df2

from elasticsearch import Elasticsearch, helpers
import configparser

config = configparser.ConfigParser()
config.read('example.ini')

es = Elasticsearch(
    cloud_id=config['ELASTIC']['cloud_id'],
    http_auth=(config['ELASTIC']['user'], config['ELASTIC']['password'])
)


if es.indices.exists(index = index + present_date):
    pass
else:
    es.indices.create(index = index + present_date, body = {
        "settings": {
            "analysis": {
                "analyzer": {
                    "nori": {
                        "tokenizer": "nori_tokenizer"
                    }
                }
            }
        },
        "mappings": {
            "properties": {
                "kyobo_subject": {
                    "type": "text",
                    "analyzer": "nori",
                    "fields":{
                        "keyword":{
                            "type":"keyword"
                        }
                    }
                },
                "kyobo_genre": {
                    "type": "keyword"
                },
                "kyobo_writer": {
                    "type": "text",
                    "analyzer": "nori",
                    "fields":{
                        "keyword":{
                            "type":"keyword"
                        }
                    }
                },
                "kyobo_publisher": {
                    "type": "text",
                    "analyzer": "nori",
                    "fields":{
                        "keyword":{
                            "type":"keyword"
                        }
                    }
                },
                "kyobo_date": {
                    "type": "text"
                },
                "smu_genre": {
                    "type": "keyword"
                },
                "kyobo_price": {
                    "type": "integer"
                },
                "kyobo_rank": {
                    "type": "integer"
                },
                "kyobo_review": {
                    "type": "float"
                },
                "present_date": {
                    "type": "date"
                }
            }
        }
    })
                      
for i in range(len(df3)) :
    es.index(index= index + present_date,
         document = {
         "kyobo_subject" : str(df3.loc[i].kyobo_subject),
         "kyobo_rank" : str(df3.loc[i].kyobo_rank),
         "kyobo_writer": str(df3.loc[i].kyobo_writer),
         "kyobo_publisher": str(df3.loc[i].kyobo_publisher),
         "kyobo_date": str(df3.loc[i].kyobo_date),
         "kyobo_price": str(df3.loc[i].kyobo_price),
         "kyobo_review": str(df3.loc[i].kyobo_review),
         "kyobo_genre": str(df3.loc[i].kyobo_genre),
         "smu_genre": str(df3.loc[i].smu_genre),
         "present_date" : present_date  
 })        








[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.78M/6.78M [00:00<00:00, 11.9MB/s]
C:\Users\woosung\AppData\Local\Temp\ipykernel_19464\1578134215.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
